In [2]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [3]:

import torch
from torch import *
dataset_x = torch.load("/kaggle/input/combined-dataset-of-grayscale-cxr-images/dataset_x.pt")
dataset_y = torch.load("/kaggle/input/combined-dataset-of-grayscale-cxr-images/dataset_y.pt")

In [4]:
from sklearn.model_selection import train_test_split
xtr, xts, ytr, yts = train_test_split(dataset_x, dataset_y, test_size=0.2, shuffle=True, stratify=dataset_y)

In [6]:
xts.shape

torch.Size([5446, 1, 224, 224])

In [14]:
import torch
from torch import *
my_dataset = torch.utils.data.TensorDataset(xtr,ytr) # create your datset
my_dataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32,
                                          shuffle=True, num_workers=2)

my_dataset = torch.utils.data.TensorDataset(xts,yts) # create your datset
my_tdataloader = torch.utils.data.DataLoader(my_dataset,batch_size=32)

# model

In [15]:
import os
import time
from tempfile import TemporaryDirectory

import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split

import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 



import timm

# from timm
pretrained_model_name = "resnet50"
model = timm.create_model(pretrained_model_name, pretrained=True)

num_channels = 1  # for grayscale images, but it could be any number
# Extract the first conv layer's parameters
num_filters = model.conv1.out_channels
kernel_size = model.conv1.kernel_size
stride = model.conv1.stride
padding = model.conv1.padding
conv1 = torch.nn.Conv2d(num_channels, num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
original_weights = model.conv1.weight.data.mean(dim=1, keepdim=True)
conv1.weight.data = original_weights.repeat(1, num_channels, 1, 1)
model.conv1 = conv1
model_ft = model

# Freeze only the convolutional layers of the pre-trained model
for param in model_ft.parameters():
    if isinstance(param, nn.Conv2d):
        param.requires_grad = False



# Modify the model head for fine-tuning
num_features = 2048

# Additional linear layer and dropout layer
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),  # Additional linear layer with 256 output features
    nn.ReLU(inplace=True),         # Activation function (you can choose other activation functions too)
    nn.Dropout(0.5),               # Dropout layer with 50% probability
    nn.Linear(256, 4)    # Final prediction fc layer
)

model_ft = torch.nn.DataParallel(model_ft, device_ids = [0,1]).to(device)
#model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)



batch_size=128
num_epochs=20
val_size = len(my_tdataloader)
train_size = len(my_dataloader) 

losses = [] 
accuracies = [] 
val_losses = [] 
val_accuracies = [] 
# Train the model 

x = range(0,num_epochs)
for epoch in x: 
	for i, (images, labels) in enumerate(my_dataloader): 
		# Forward pass 
		images=images.to(device) 
		labels=labels.type(torch.LongTensor).to(device) 
		outputs = model(images) 
		loss = criterion(outputs, labels) 
		
		# Backward pass and optimization 
		optimizer_ft.zero_grad() 
		loss.backward() 
		optimizer_ft.step() 

		_, predicted = torch.max(outputs.data, 1) 
	acc = (predicted == labels).sum().item() / labels.size(0) 
	accuracies.append(acc) 
	losses.append(loss.item()) 

	# Evaluate the model on the validation set	
	print('Epoch [{}/{}],Loss:{:.4f},Accuracy:{:.2f}'.format( 
		epoch+1, num_epochs, loss.item(),acc))







/tmp/ipykernel_26/315989530.py:81: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  x = range(0,num_epochs)


Epoch [1.0/20],Loss:1.3075,Accuracy:0.41


KeyboardInterrupt: 